# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116195e+02     1.571076e+00
 * time: 0.12368917465209961
     1     1.001226e+01     9.591430e-01
 * time: 1.3406221866607666
     2    -1.252381e+01     1.044366e+00
 * time: 1.48313307762146
     3    -3.480647e+01     7.511850e-01
 * time: 1.6419041156768799
     4    -4.834769e+01     5.134034e-01
 * time: 1.779414176940918
     5    -5.729449e+01     2.086142e-01
 * time: 1.9295930862426758
     6    -6.001588e+01     7.823043e-02
 * time: 2.0459370613098145
     7    -6.095000e+01     5.507601e-02
 * time: 2.1542630195617676
     8    -6.131609e+01     6.578038e-02
 * time: 2.270153045654297
     9    -6.162230e+01     3.796409e-02
 * time: 2.3875250816345215
    10    -6.181544e+01     2.639569e-02
 * time: 2.499983072280884
    11    -6.195438e+01     2.078613e-02
 * time: 2.617824077606201
    12    -6.202365e+01     1.679799e-02
 * time: 2.72527813911438
    13    -6.209244e+01     1.406445e-02
 * time: 2.84051513671875
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557689
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666462335
